## 4.1 将LLM接入LangChain

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('.env.local'))

api_key = os.getenv("DEEPSEEK_API_KEY")
api_url = os.getenv("DEEPSEEK_API_URL")
model = os.getenv("DEEPSEEK_MODEL")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key,
    base_url=api_url,
    model_name=model,
    temperature=0.0,
)

/Users/young/project/llmProject/llm-universe/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
output = llm.invoke("请你自我介绍一下自己！")

In [5]:
output

AIMessage(content='你好呀！我是 **DeepSeek Chat**，由深度求索（DeepSeek）公司打造的智能 AI 助手。我的最新版本是 **DeepSeek-V3**，知识截止到 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读和解析 **PDF、Word、Excel、PPT、TXT** 等文件内容。  \n\n### **我的特点：**  \n🔹 **免费使用**：目前不收费，随时为你解答问题！  \n🔹 **超长上下文**：支持长达 128K 的对话记忆，适合处理复杂任务。  \n🔹 **文件阅读**：可以帮你分析文档内容，提取关键信息。  \n🔹 **知识丰富**：覆盖科技、编程、学习、生活、娱乐等多个领域。  \n🔹 **逻辑清晰**：擅长推理、写作、翻译、代码编写等任务。  \n\n无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊 你今天有什么想了解的呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 8, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'finish_reason': 'stop', 'logprobs': None}, id='run-1553582b-f98b-473b-9a15-1a507ab9ad7b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 218, 'total_tokens': 226})

In [8]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

In [9]:
# 使用prompt模板
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template)
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"

messages = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})])

In [10]:
# 测试一下
output = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 45, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat'

In [11]:
# 格式化输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)'

In [12]:
# 使用LCEL表达式
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language": "中文", "output_language": "英文", "text": text})

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.'

In [13]:
# 再试一下，英译中
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比身体还重的行李潜入尼罗河底，穿过几道闪电后看见一堆光晕，不知是不是这里。'

## 4.2 构建检索问答链